In [ ]:
# global packages
%matplotlib inline
import tensorflow as tf
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from os import path

In [ ]:
# local packages
import sys
sys.path.append('../')
from src.convolutional import *
from src.make_branch_alexnet import *

In [ ]:
# net to import
net_path = '../pretrained/xc_conv5_32c_s4_e40.mat'
# Layers that need to be imported from MatConvnet architecture
conv_to_import = ['br1_conv1','br1_conv2','br1_conv3','br1_conv4','br1_conv5','br2_conv1','br2_conv2','br2_conv3','br2_conv4','br2_conv5']
bnorm_to_import = ['br1_bn1','br1_bn2','br1_bn3','br1_bn4','br2_bn1','br2_bn2','br2_bn3','br2_bn4','fin_adjust_bn']
layers_to_import = conv_to_import + bnorm_to_import

# read mat file from path
mat = scipy.io.loadmat(net_path)
## This is for .mat files saved with --v7.3
# import h5py
# net_file = h5py.File(net_path, 'r')
# data = f.get('...') 
net = mat.get('net')

In [ ]:
layers = net['layers']
layers = layers[0][0]
# all layer names
layer_names = layers['name'][0]
# copy the numpy.ndarray in a list
layer_names_list = [layer_names[l][0] for l in xrange(layer_names.size)]
# all param names organized by layers
param_names = layers['params'][0]
# param_names_list = [param_names[l][0] for l in xrange(param_names.size)]

# verify that layers_to_import are all present in layer_names + find positions
layers_to_import_pos = [layer_names_list.index(lname) for lname in layers_to_import]

#TODO: raise error if in mat file there are layers with learnt parameters not present in layers_to_import

# create a dictionary to map layer -> params (<key,value> = <string,list> )
layer_to_params = {}
for i in xrange(len(layers_to_import_pos)):    
    l_param_names = param_names[layers_to_import_pos[i]][0]
    l_param_names_list = [l_param_names[p][0] for p in xrange(l_param_names.size)]
    layer_to_params[layer_names_list[layers_to_import_pos[i]]] = l_param_names_list    

In [ ]:
# organize parameters to import
params = net['params']
params = params[0][0]
params_names = params['name'][0]
params_names_list = [params_names[p][0] for p in xrange(params_names.size)]
params_values = params['value'][0]
params_values_list = [params_values[p] for p in xrange(params_values.size)]

In [ ]:
# layers_to_import -> layers_to_params -> params_names_list -> params_values_list

# e.g:
conv1f = params_values_list[params_names_list.index('br_conv1f')]
conv1b = params_values_list[params_names_list.index('br_conv1b')]
X = tf.placeholder(tf.float32)
X1 = tf.expand_dims(X, 0)
#TODO: function convolutional does not know tf
#h1 = set_convolutional(X1, conv1f, conv1b, 11, 11, 3, 96, stride=2, scope='conv1')


In [ ]:
# root_path = '/media/berti/STORAGE1/ILSVRC2015_crops/Data/VID/train'
# video_path = 'b/ILSVRC2015_train_00234012'
# crop = '000019.01.crop.z.jpg'
# im = plt.imread(path.join(root_path, video_path, crop))
# print im.shape
# plt.imshow(im)

In [ ]:
# with tf.Session() as sess:
#     sess.run(tf.initialize_all_variables())
#     out = sess.run(h1, feed_dict={X: im})

# print(out.shape)

In [ ]:
# tf.InteractiveSession()
